In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Helper functions creation area

In [2]:
def getLatitude(bairro):
    address = bairro +', São Paulo'
    geolocator = Nominatim(user_agent="Sao Paulo Explorer")
    lat=0
    try:    
        location = geolocator.geocode(address)
        lat = location.latitude
    except:
        getLatitude(bairro)
    if (lat == 0.000000):
        getLatitude(bairro)        
    return lat

def getLongitude(bairro):
    address = bairro +', São Paulo'
    geolocator = Nominatim(user_agent="Sao Paulo Explorer")
    lon = 0 
    try:
      location = geolocator.geocode(address)
      lon = location.longitude 
    except:
        getLongitude(bairro)      
    if (lon == 0.000000):
        getLongitude(bairro)      
    return lon

def generateUrl(neighborhood_latitude, neighborhood_longitude):
  # type your answer here
  LIMIT = 100 # limit of number of venues returned by Foursquare API

  radius = 500 # define radius

  categoryId = '4d4b7105d754a06374d81259'

  # create URL
  url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT,
    categoryId) #CategoryId for food
  #print(url)
  return url

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Setting Foursquare credentials

In [5]:
CLIENT_ID = 'XMMI1Q3UBKQFRZDGUDICLOYRFSD3TVPWVD14ML154NYYMN0W' # your Foursquare ID
CLIENT_SECRET = 'RKCYA3MSWT2TA5ZWFHG3AQUHEUXJ3YN5UFAD5K30ECGTYLFU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XMMI1Q3UBKQFRZDGUDICLOYRFSD3TVPWVD14ML154NYYMN0W
CLIENT_SECRET:RKCYA3MSWT2TA5ZWFHG3AQUHEUXJ3YN5UFAD5K30ECGTYLFU


Creating a dataframe with the neigborhoods of Sao Paulo and their Latitude and Longitude

In [3]:
table = pd.read_html('https://pt.wikipedia.org/wiki/Lista_dos_distritos_de_S%C3%A3o_Paulo_por_popula%C3%A7%C3%A3o',
                      index_col=0, header=0, match='Distrito')
saoPaulo = table[0]
saoPaulo.reset_index(inplace=True)
saoPaulo.drop(columns={'Posição','População 2010','Censo 2000'},inplace=True)
saoPaulo['Latitude']  = saoPaulo['Distrito'].apply(lambda x: getLatitude(x))
saoPaulo['Longitude'] = saoPaulo['Distrito'].apply(lambda x: getLongitude(x))
saoPaulo.head()

,Distrito,Latitude,Longitude
0,Grajaú,-23.786248,-46.664993
1,Sapopemba,-23.604559,-46.509823
2,Jardim Ângela,-23.712246,-46.771206
3,Brasilândia,-23.448272,-46.690269
4,Capão Redondo,-23.671903,-46.779435


Querying at Foursquare the Food Places of Sao Paulo by Neighborhood (Distrito)

In [31]:
bairros = saoPaulo.Distrito.unique()
#bairros=['Chinatown','Inwood']
saoPaulo_food = pd.DataFrame()
for neigh in bairros:
  neighbor = saoPaulo[saoPaulo['Distrito']==neigh]
  neighborhood_latitude = neighbor['Latitude'].values[0] # neighborhood latitude value
  neighborhood_longitude = neighbor['Longitude'].values[0] # neighborhood longitude value
  neighborhood_name = neigh # neighborhood name
  url = generateUrl(neighborhood_latitude, neighborhood_longitude)
  results = requests.get(url).json()
  
  venues = results['response']['groups'][0]['items']
    
  nearby_venues = pd.json_normalize(venues) # flatten JSON

  # filter columns
  filtered_columns = ['venue.categories']
  if (not nearby_venues.empty):
      nearby_venues =nearby_venues.loc[:, filtered_columns]

      # filter the category for each row
      nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

      # clean columns
      nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
      group = nearby_venues['categories'].value_counts()
      group = pd.DataFrame(group)
      group = group.sort_values(by=['categories'],ascending=False)
      #group = group.head(10)
      group = group.transpose()
      group.rename(index={'categories':neigh},inplace=True)
      group.reset_index(inplace=True)
      group.rename(columns={'index':'Distrito'},inplace=True)
      if (saoPaulo_food.empty):
        saoPaulo_food = group
      else:
        saoPaulo_food = saoPaulo_food.append(group)
    
saoPaulo_food = saoPaulo_food.fillna(0)
saoPaulo_food.head()

,Distrito,Gastropub,Food,Breakfast Spot,Falafel Restaurant,Pastelaria,Japanese Restaurant,Food Truck,Pizza Place,Sandwich Place,Bakery,Diner,Steakhouse,Snack Place,BBQ Joint,Fried Chicken Joint,Restaurant,South American Restaurant,Brazilian Restaurant,Café,Deli / Bodega,Fast Food Restaurant,Burger Joint,Bagel Shop,Italian Restaurant,Hot Dog Joint,Sushi Restaurant,Buffet,Asian Restaurant,Churrascaria,Comfort Food Restaurant,Portuguese Restaurant,Fish & Chips Shop,Chinese Restaurant,Northeastern Brazilian Restaurant,Bistro,Mineiro Restaurant,Vegetarian / Vegan Restaurant,Southeastern Brazilian Restaurant,Middle Eastern Restaurant,Argentinian Restaurant,Mexican Restaurant,Thai Restaurant,Empada House,Halal Restaurant,Donburi Restaurant,American Restaurant,Cafeteria,Wings Joint,Baiano Restaurant,Spanish Restaurant,Salad Place,African Restaurant,Latin American Restaurant,Seafood Restaurant,Doner Restaurant,Kebab Restaurant,Acai House,Paella Restaurant,Empanada Restaurant,Burrito Place,Food Stand,Food Court,Greek Restaurant,Tapas Restaurant,French Restaurant,Lebanese Restaurant,Turkish Restaurant,Poke Place,Trattoria/Osteria,Southern Brazilian Restaurant,Taco Place,Noodle House,Korean Restaurant,German Restaurant,Swiss Restaurant,Tapiocaria,Northern Brazilian Restaurant,Vietnamese Restaurant,Indian Restaurant,Peruvian Restaurant,Piadineria,Mediterranean Restaurant,Shawarma Place,Arepa Restaurant,Jewish Restaurant,Borek Place
0,Sapopemba,2.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,Jardim Ângela,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,Brasilândia,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,Capão Redondo,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,Jardim São Luís,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
print('Sao Paulo has {} different food places categories registered at Foursquare in {} neighborhoods'.format(len(saoPaulo_food.columns)-1,len(saoPaulo_food.index) + 1))

Sao Paulo has 86 different food places registered at Foursquare in 91 neighborhoods


In [26]:
neighborhoodsWithVenues = saoPaulo_food.Distrito.unique()
neighWithoutVenues = saoPaulo[~saoPaulo.Distrito.isin(neighborhoodsWithVenues)]
neighborhoodsWithoutVenues = neighWithoutVenues.Distrito.unique()
bairros = ''
quant = len(neighWithoutVenues.index)-1
x=0
for bairro in neighborhoodsWithoutVenues:
    x = x + 1
    if (x < quant):
      bairros = bairros + bairro + ', '
    elif (x==quant):
      bairros = bairros + bairro + ' and '  
    else:
      bairros = bairros + bairro + '.'  
print('There are {} neighborhoods without venues registered at Foursquare, which are {}'.format(len(neighWithoutVenues.index),bairros))

There are 6 neighborhoods without venues registered at Foursquare, which are Grajaú, Tremembé, Cachoeirinha, São Rafael, Cidade Líder and Marsilac.


In [57]:
saoPaulo_food_Count = pd.DataFrame(saoPaulo_food)
saoPaulo_food_Count['Index'] = 'Sao Paulo'
saoPaulo_food_Count.drop(columns=('Distrito'),inplace=True)
saoPaulo_food_Count = saoPaulo_food_Count.groupby('Index').sum()
saoPaulo_food_Count = saoPaulo_food_Count.transpose()
saoPaulo_food_Count.reset_index(inplace=True)
saoPaulo_food_Count.rename(columns={'index':'Venue','Sao Paulo':'Quantity'}, inplace=True)
saoPaulo_food_Count = saoPaulo_food_Count.sort_values(by=('Quantity'),ascending=False)
print('The 10 most common food places in the whole city are:')
saoPaulo_food_Count.head(10)

The 10 most common food places in the whole city are:


Index,Venue,Quantity
17,Brazilian Restaurant,244.0
9,Bakery,175.0
15,Restaurant,172.0
7,Pizza Place,168.0
18,Café,122.0
21,Burger Joint,99.0
23,Italian Restaurant,78.0
5,Japanese Restaurant,71.0
12,Snack Place,71.0
6,Food Truck,53.0


Calculating the most common food places by neighborhood

In [62]:
saoPaulo_food.drop(columns={'City','Index'},inplace=True)
num_top_venues = 10

for hood in saoPaulo_food['Distrito']:
    #print("----"+hood+"----")
    temp = saoPaulo_food[saoPaulo_food['Distrito'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    #print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))

    num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Distrito']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
foodPlaces_sorted = pd.DataFrame(columns=columns)
foodPlaces_sorted['Distrito'] = saoPaulo_food['Distrito']

for ind in np.arange(saoPaulo_food.shape[0]):
    foodPlaces_sorted.iloc[ind, 1:] = return_most_common_venues(saoPaulo_food.iloc[ind, :], num_top_venues)

foodPlaces_sorted.head()

,Distrito,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Sapopemba,Gastropub,Food,Breakfast Spot,Falafel Restaurant,Pastelaria,Japanese Restaurant,Chinese Restaurant,Sushi Restaurant,Buffet,Asian Restaurant
0,Jardim Ângela,Pastelaria,Borek Place,Fish & Chips Shop,Italian Restaurant,Hot Dog Joint,Sushi Restaurant,Buffet,Asian Restaurant,Churrascaria,Comfort Food Restaurant
0,Brasilândia,Food Truck,Pizza Place,Borek Place,Chinese Restaurant,Hot Dog Joint,Sushi Restaurant,Buffet,Asian Restaurant,Churrascaria,Comfort Food Restaurant
0,Capão Redondo,Food Truck,Pizza Place,Sandwich Place,Bakery,Diner,Borek Place,Chinese Restaurant,Sushi Restaurant,Buffet,Asian Restaurant
0,Jardim São Luís,Snack Place,Japanese Restaurant,Pizza Place,Steakhouse,Borek Place,Sushi Restaurant,Buffet,Asian Restaurant,Churrascaria,Comfort Food Restaurant


Generating 15 cluster to aggreagate the neighborhoods with common food places

In [63]:
# set number of clusters
kclusters = 15

foodPlaces_grouped_clustering = saoPaulo_food.drop('Distrito', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(foodPlaces_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10] )


# add clustering labels
foodPlaces_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

saoPaulo_common_foodPlaces = saoPaulo

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
saoPaulo_common_foodPlaces = pd.merge(saoPaulo_common_foodPlaces, foodPlaces_sorted.set_index('Distrito'), on='Distrito')
saoPaulo_common_foodPlaces['Cluster Labels'] = saoPaulo_common_foodPlaces['Cluster Labels'].astype(int)
saoPaulo_common_foodPlaces.head() # check the last columns!

[0 0 0 0 0 0 0 5 0 0]


,Distrito,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Sapopemba,-23.604559,-46.509823,0,Gastropub,Food,Breakfast Spot,Falafel Restaurant,Pastelaria,Japanese Restaurant,Chinese Restaurant,Sushi Restaurant,Buffet,Asian Restaurant
1,Jardim Ângela,-23.712246,-46.771206,0,Pastelaria,Borek Place,Fish & Chips Shop,Italian Restaurant,Hot Dog Joint,Sushi Restaurant,Buffet,Asian Restaurant,Churrascaria,Comfort Food Restaurant
2,Brasilândia,-23.448272,-46.690269,0,Food Truck,Pizza Place,Borek Place,Chinese Restaurant,Hot Dog Joint,Sushi Restaurant,Buffet,Asian Restaurant,Churrascaria,Comfort Food Restaurant
3,Capão Redondo,-23.671903,-46.779435,0,Food Truck,Pizza Place,Sandwich Place,Bakery,Diner,Borek Place,Chinese Restaurant,Sushi Restaurant,Buffet,Asian Restaurant
4,Jardim São Luís,-23.683573,-46.737762,0,Snack Place,Japanese Restaurant,Pizza Place,Steakhouse,Borek Place,Sushi Restaurant,Buffet,Asian Restaurant,Churrascaria,Comfort Food Restaurant


Obtaining Sao Paulo coordinates

In [65]:
address = 'São Paulo, São Paulo'

geolocator = Nominatim(user_agent="Sao Paulo Explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of São Paulo are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of São Paulo are -23.5506507, -46.6333824.


Generating Sao Paulo map with the clusters information

In [66]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11,tiles="OpenStreetMap")

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(saoPaulo_common_foodPlaces['Latitude'], saoPaulo_common_foodPlaces['Longitude'], saoPaulo_common_foodPlaces['Distrito'], saoPaulo_common_foodPlaces['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Cluster 1

In [70]:
saoPaulo_common_foodPlaces.loc[saoPaulo_common_foodPlaces['Cluster Labels'] == 0, saoPaulo_common_foodPlaces.columns[[0] + list(range(4, saoPaulo_common_foodPlaces.shape[1]))]]

,Distrito,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Sapopemba,Gastropub,Food,Breakfast Spot,Falafel Restaurant,Pastelaria,Japanese Restaurant,Chinese Restaurant,Sushi Restaurant,Buffet,Asian Restaurant
1,Jardim Ângela,Pastelaria,Borek Place,Fish & Chips Shop,Italian Restaurant,Hot Dog Joint,Sushi Restaurant,Buffet,Asian Restaurant,Churrascaria,Comfort Food Restaurant
2,Brasilândia,Food Truck,Pizza Place,Borek Place,Chinese Restaurant,Hot Dog Joint,Sushi Restaurant,Buffet,Asian Restaurant,Churrascaria,Comfort Food Restaurant
3,Capão Redondo,Food Truck,Pizza Place,Sandwich Place,Bakery,Diner,Borek Place,Chinese Restaurant,Sushi Restaurant,Buffet,Asian Restaurant
4,Jardim São Luís,Snack Place,Japanese Restaurant,Pizza Place,Steakhouse,Borek Place,Sushi Restaurant,Buffet,Asian Restaurant,Churrascaria,Comfort Food Restaurant
5,Cidade Ademar,BBQ Joint,Bakery,Steakhouse,Brazilian Restaurant,Pizza Place,South American Restaurant,Restaurant,Fried Chicken Joint,Asian Restaurant,Churrascaria
6,Itaim Paulista,Bakery,Steakhouse,Pastelaria,Food Truck,Pizza Place,Northeastern Brazilian Restaurant,Sushi Restaurant,Buffet,Asian Restaurant,Churrascaria
8,Jaraguá,Bakery,Italian Restaurant,Brazilian Restaurant,Pizza Place,Fish & Chips Shop,Hot Dog Joint,Sushi Restaurant,Buffet,Asian Restaurant,Churrascaria
9,Cidade Tiradentes,Burger Joint,Steakhouse,Restaurant,Pizza Place,Hot Dog Joint,Sushi Restaurant,Buffet,Asian Restaurant,Churrascaria,Comfort Food Restaurant
10,Campo Limpo,Food Truck,Pizza Place,Restaurant,Diner,Fried Chicken Joint,Borek Place,Fish & Chips Shop,Sushi Restaurant,Buffet,Asian Restaurant


Cluster 2

In [71]:
saoPaulo_common_foodPlaces.loc[saoPaulo_common_foodPlaces['Cluster Labels'] == 1, saoPaulo_common_foodPlaces.columns[[0] + list(range(4, saoPaulo_common_foodPlaces.shape[1]))]]

,Distrito,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
60,Tatuapé,Brazilian Restaurant,Italian Restaurant,Pastelaria,Pizza Place,Fast Food Restaurant,Café,BBQ Joint,Burger Joint,Food Truck,Steakhouse


Cluster 3 

In [72]:
saoPaulo_common_foodPlaces.loc[saoPaulo_common_foodPlaces['Cluster Labels'] == 2, saoPaulo_common_foodPlaces.columns[[0] + list(range(4, saoPaulo_common_foodPlaces.shape[1]))]]

,Distrito,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
87,Sé,Brazilian Restaurant,Café,Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Snack Place,Fast Food Restaurant,Breakfast Spot,American Restaurant


Cluster 4

In [73]:
saoPaulo_common_foodPlaces.loc[saoPaulo_common_foodPlaces['Cluster Labels'] == 3, saoPaulo_common_foodPlaces.columns[[0] + list(range(4, saoPaulo_common_foodPlaces.shape[1]))]]

,Distrito,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
40,Perdizes,Burger Joint,Restaurant,Café,Pizza Place,Bakery,Brazilian Restaurant,Snack Place,Middle Eastern Restaurant,Italian Restaurant,Deli / Bodega
66,Campo Belo,Bakery,Brazilian Restaurant,Pizza Place,Restaurant,Café,Fast Food Restaurant,Burger Joint,Italian Restaurant,BBQ Joint,German Restaurant


Cluster 5

In [74]:
saoPaulo_common_foodPlaces.loc[saoPaulo_common_foodPlaces['Cluster Labels'] == 4, saoPaulo_common_foodPlaces.columns[[0] + list(range(4, saoPaulo_common_foodPlaces.shape[1]))]]

,Distrito,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
84,Bom Retiro,Korean Restaurant,Brazilian Restaurant,Chinese Restaurant,Café,Restaurant,Bakery,Diner,Snack Place,Japanese Restaurant,Borek Place


Cluster 6

In [75]:
saoPaulo_common_foodPlaces.loc[saoPaulo_common_foodPlaces['Cluster Labels'] == 5, saoPaulo_common_foodPlaces.columns[[0] + list(range(4, saoPaulo_common_foodPlaces.shape[1]))]]

,Distrito,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Sacomã,Brazilian Restaurant,Restaurant,Fast Food Restaurant,Diner,Deli / Bodega,Café,Hot Dog Joint,Italian Restaurant,Bagel Shop,Burger Joint
56,Itaim Bibi,Restaurant,Brazilian Restaurant,Churrascaria,Café,Sushi Restaurant,Middle Eastern Restaurant,Japanese Restaurant,Food Truck,Italian Restaurant,Bakery
68,Santo Amaro,Restaurant,Japanese Restaurant,Snack Place,Italian Restaurant,Burger Joint,Food Truck,Sandwich Place,Bakery,Mineiro Restaurant,Bistro
81,Morumbi,Restaurant,Snack Place,Café,Food Truck,Brazilian Restaurant,Food,Borek Place,Buffet,Asian Restaurant,Churrascaria
85,Brás,Brazilian Restaurant,Restaurant,Bakery,Diner,Buffet,Bagel Shop,Italian Restaurant,Burger Joint,Vegetarian / Vegan Restaurant,Steakhouse
88,Pari,Middle Eastern Restaurant,Brazilian Restaurant,Restaurant,Bakery,Italian Restaurant,Bagel Shop,Fast Food Restaurant,Kebab Restaurant,Pizza Place,Café


Cluster 7

In [76]:
saoPaulo_common_foodPlaces.loc[saoPaulo_common_foodPlaces['Cluster Labels'] == 6, saoPaulo_common_foodPlaces.columns[[0] + list(range(4, saoPaulo_common_foodPlaces.shape[1]))]]

,Distrito,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
75,Consolação,Brazilian Restaurant,Café,Burger Joint,Pizza Place,Vegetarian / Vegan Restaurant,Bakery,Restaurant,Food Truck,Mineiro Restaurant,Italian Restaurant
76,República,Brazilian Restaurant,Café,Restaurant,Burger Joint,Pizza Place,Vegetarian / Vegan Restaurant,Italian Restaurant,Fast Food Restaurant,BBQ Joint,Sandwich Place


Cluster 8

In [77]:
saoPaulo_common_foodPlaces.loc[saoPaulo_common_foodPlaces['Cluster Labels'] == 7, saoPaulo_common_foodPlaces.columns[[0] + list(range(4, saoPaulo_common_foodPlaces.shape[1]))]]

,Distrito,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
71,Bela Vista,Italian Restaurant,Brazilian Restaurant,Pizza Place,Café,Burger Joint,Restaurant,Japanese Restaurant,Bakery,Sushi Restaurant,Steakhouse


Cluster 9

In [78]:
saoPaulo_common_foodPlaces.loc[saoPaulo_common_foodPlaces['Cluster Labels'] == 8, saoPaulo_common_foodPlaces.columns[[0] + list(range(4, saoPaulo_common_foodPlaces.shape[1]))]]

,Distrito,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Jardim Helena,Food Truck,Pizza Place,Bakery,Burger Joint,Brazilian Restaurant,Comfort Food Restaurant,Restaurant,Borek Place,Fish & Chips Shop,Buffet
23,Freguesia do Ó,Bakery,Pizza Place,Steakhouse,Asian Restaurant,Pastelaria,Japanese Restaurant,Sandwich Place,Portuguese Restaurant,Snack Place,Northeastern Brazilian Restaurant
24,São Lucas,Brazilian Restaurant,Restaurant,Steakhouse,Food Truck,Bakery,Chinese Restaurant,Pastelaria,Pizza Place,Café,Fish & Chips Shop
27,Penha,Brazilian Restaurant,Japanese Restaurant,Restaurant,Pizza Place,Gastropub,Bakery,Italian Restaurant,Burger Joint,Deli / Bodega,BBQ Joint
33,Saúde,Pizza Place,Bakery,Brazilian Restaurant,Restaurant,Bagel Shop,Japanese Restaurant,Food Truck,Sandwich Place,Diner,Café
36,Artur Alvim,Snack Place,Asian Restaurant,Pizza Place,Bakery,Cafeteria,Hot Dog Joint,Chinese Restaurant,Fast Food Restaurant,Deli / Bodega,Brazilian Restaurant
39,Vila Matilde,Pizza Place,Bakery,Steakhouse,Brazilian Restaurant,Salad Place,Diner,Restaurant,Chinese Restaurant,Buffet,Asian Restaurant
42,Campo Grande,Bakery,Brazilian Restaurant,Restaurant,Diner,Snack Place,Food,Food Truck,Pizza Place,Steakhouse,Asian Restaurant
43,Vila Prudente,Pizza Place,Brazilian Restaurant,Bakery,Asian Restaurant,Burger Joint,Pastelaria,Snack Place,BBQ Joint,Acai House,Deli / Bodega
48,Cursino,Bakery,Pizza Place,Brazilian Restaurant,Gastropub,Fish & Chips Shop,Hot Dog Joint,Sushi Restaurant,Buffet,Asian Restaurant,Churrascaria


Cluster 10

In [79]:
saoPaulo_common_foodPlaces.loc[saoPaulo_common_foodPlaces['Cluster Labels'] == 9, saoPaulo_common_foodPlaces.columns[[0] + list(range(4, saoPaulo_common_foodPlaces.shape[1]))]]

,Distrito,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
58,Jardim Paulista,Italian Restaurant,Restaurant,Brazilian Restaurant,Japanese Restaurant,Bakery,Greek Restaurant,Pizza Place,Spanish Restaurant,Café,Burger Joint


Cluster 11

In [80]:
saoPaulo_common_foodPlaces.loc[saoPaulo_common_foodPlaces['Cluster Labels'] == 10, saoPaulo_common_foodPlaces.columns[[0] + list(range(4, saoPaulo_common_foodPlaces.shape[1]))]]

,Distrito,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
67,Pinheiros,Brazilian Restaurant,Restaurant,Japanese Restaurant,Burger Joint,Café,Diner,Buffet,Snack Place,Comfort Food Restaurant,Acai House


Cluster 12

In [81]:
saoPaulo_common_foodPlaces.loc[saoPaulo_common_foodPlaces['Cluster Labels'] == 11, saoPaulo_common_foodPlaces.columns[[0] + list(range(4, saoPaulo_common_foodPlaces.shape[1]))]]

,Distrito,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,Santa Cecília,Brazilian Restaurant,Restaurant,Japanese Restaurant,Pizza Place,Bakery,Café,Sushi Restaurant,Bistro,BBQ Joint,Asian Restaurant
89,Barra Funda,Brazilian Restaurant,Restaurant,Japanese Restaurant,Café,Sandwich Place,Snack Place,Food Stand,Italian Restaurant,Food Truck,Pizza Place


Cluster 13

In [82]:
saoPaulo_common_foodPlaces.loc[saoPaulo_common_foodPlaces['Cluster Labels'] == 12, saoPaulo_common_foodPlaces.columns[[0] + list(range(4, saoPaulo_common_foodPlaces.shape[1]))]]

,Distrito,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Vila Mariana,Brazilian Restaurant,Restaurant,Pizza Place,Bakery,Snack Place,Burger Joint,Café,Japanese Restaurant,Steakhouse,Deli / Bodega
31,Santana,Brazilian Restaurant,Restaurant,Fast Food Restaurant,Burger Joint,Pizza Place,Middle Eastern Restaurant,Bakery,Mexican Restaurant,Japanese Restaurant,Food Truck
69,Lapa,Brazilian Restaurant,Restaurant,Pastelaria,Pizza Place,Bakery,Fast Food Restaurant,Japanese Restaurant,Café,Snack Place,Italian Restaurant
70,Liberdade,Pizza Place,Snack Place,Brazilian Restaurant,Chinese Restaurant,Bakery,Steakhouse,Restaurant,Food Truck,BBQ Joint,Pastelaria
82,Vila Leopoldina,Brazilian Restaurant,Bakery,Burger Joint,Snack Place,Restaurant,Pizza Place,Italian Restaurant,Peruvian Restaurant,Japanese Restaurant,Sandwich Place


Cluster 14

In [83]:
saoPaulo_common_foodPlaces.loc[saoPaulo_common_foodPlaces['Cluster Labels'] == 13, saoPaulo_common_foodPlaces.columns[[0] + list(range(4, saoPaulo_common_foodPlaces.shape[1]))]]

,Distrito,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,Tucuruvi,Fast Food Restaurant,Brazilian Restaurant,Pizza Place,Snack Place,Bakery,Sandwich Place,Fried Chicken Joint,Italian Restaurant,Hot Dog Joint,Café


Cluster 15

In [84]:
saoPaulo_common_foodPlaces.loc[saoPaulo_common_foodPlaces['Cluster Labels'] == 14, saoPaulo_common_foodPlaces.columns[[0] + list(range(4, saoPaulo_common_foodPlaces.shape[1]))]]

,Distrito,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Ipiranga,Burger Joint,Pizza Place,Brazilian Restaurant,Bakery,Italian Restaurant,Middle Eastern Restaurant,Bistro,Sushi Restaurant,Bagel Shop,Café
65,Mooca,Burger Joint,Japanese Restaurant,Pizza Place,Mexican Restaurant,Bakery,BBQ Joint,Restaurant,Middle Eastern Restaurant,Italian Restaurant,Food Truck


Sao Paulo is a very diverse city with a population with many options of restaurants where you can find almost everything. The cluster generated by the similarity of food places, reflects how the city had grown through time with different waves of imigirations and emigrations.
And by the distribution of cluster you can easily see where they are placed.
